In [1]:
import os as os
import requests as req
import zipfile as zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def data_directories():
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    """

    # Diretorios para armazenamento dos dados coletados
    directories = ['..\\data\\01-collected'
                  ,'..\\data\\02-cleaned'
                  ,'..\\data\\03-organized'
                  ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados iniciada.')
    
    # Para cada diretorio na lista de diretorios
    for directory in directories:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados finalizada.')

In [3]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo. Ao final do download, e verificado
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado. Caso contrario,
    o arquivo e descompactado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)
    
    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Log: Mensagem de URL encontrada e de inicio do download
            print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo csv 
        # na pasta e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])
            
    else:
        print(str(datetime.now()) + ': Nao foram encontrados arquivos referentes ao ano de ' + kwargs['file_date_ref'])

In [6]:
def collect_data(**kwargs):
    """
    Coleta os dados para cada ano de referencia dentro das datas de inicio e fim informadas.
    
    Args:
        dt_ini_ref (date): Data inicial para coleta dos dados
        dt_fim_ref (date): Data final para coleta dos dados
    """
    
    # Cria os diretorios para armazenamento dos dados
    data_directories()
    
    # Define o diretorio para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    # Define a URL base de busca dos arquivos para download
    kwargs['download_url'] = 'http://arquivos.prf.gov.br/arquivos/index.php/s/'
    
    # Define o nome e extensao do arquivo para download
    kwargs['file_names'] = ['jdDLrQIf33xXSCe'  # Referente a 2020
                           ,'kRBUylqz6DyQznN'  # Referente a 2019
                           ,'MaC6cieXSFACNWT'  # Referente a 2018
                           ,'nqvFu7xEF6HhbAq'  # Referente a 2017
                           ] 
    kwargs['file_ext'] = '.zip'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos iniciada.')
       
    # Para cada mes de referencia dentro das datas de inicio e fim
    for file in kwargs['file_names']:
        
        # Prepara as variaveis do mes de referencia
        kwargs['download_file'] = file + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = kwargs['download_url'] + file + '/download'
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)
    
    # Log: Mensagem de inicio da remocao de arquivos nao relacionados com os dados
    print(str(datetime.now()) + ': Verificando a existencia de arquivos nao relacionados com os dados.')
    
    # Remove arquivos nao relacionados com os dados (Ex.: leiame.pdf)
    downloaded_files = os.listdir(kwargs['download_directory'])
    unrelated_files = [file for file in downloaded_files if file.endswith('.pdf')]
    for file in unrelated_files:
        file_to_remove = os.path.join(kwargs['download_directory'], file)
        os.remove(file_to_remove)
        
        # Log: Mensagem de remocao de arquivo nao relacionado com os dados
        print(str(datetime.now()) + ': Arquivo ' + file_to_remove + ' removido.')
        
    # Log: Mensagem de fim da remocao de arquivos nao relacionados com os dados
    print(str(datetime.now()) + ': Remocao dos arquivos nao relacionados com os dados concluida.')

    # Log: Mensagem de finalizacao do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos finalizada.')

In [7]:
def main():
    """
    Coleta os dados de acidentes por ocorrencia nas rodovias federais. 
    Origem: Policia Rodoviaria Federal (PRF)
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2021-05-30 21:28:40.015309: Criacao dos diretorios para armazenamento dos dados iniciada.
2021-05-30 21:28:40.016306: Diretorio ..\data\01-collected ja existe.
2021-05-30 21:28:40.016306: Diretorio ..\data\02-cleaned ja existe.
2021-05-30 21:28:40.016306: Diretorio ..\data\03-organized ja existe.
2021-05-30 21:28:40.016306: Diretorio ..\data\04-standardized ja existe.
2021-05-30 21:28:40.017304: Criacao dos diretorios para armazenamento dos dados finalizada.
2021-05-30 21:28:40.017304: Coleta dos arquivos iniciada.
2021-05-30 21:28:40.017304: Inicio do download do arquivo ..\data\01-collected\jdDLrQIf33xXSCe.zip
2021-05-30 21:28:40.298118: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\jdDLrQIf33xXSCe.zip
2021-05-30 21:28:40.633190: Descompactando o arquivo ..\data\01-collected\jdDLrQIf33xXSCe.zip
2021-05-30 21:28:40.717693: Fim do download do arquivo ..\data\01-collected\jdDLrQIf33xXSCe.zip
2021-05-30 21:28:40.718659: Inicio do download do arquivo ..\data\01-colle